Test how to read the different images within a path and extract the meta-data.

In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
load_dotenv(override=True)


True

In [ ]:
from datetime import datetime

def create_image_metadata_dict(folder: Path) -> dict:
    image_metadata_dict = {}
    for file in folder.iterdir():
        if file.is_dir():
            nested_dict = create_image_metadata_dict(file)
            image_metadata_dict.update(nested_dict)
        else:
            if file.suffix.lower() in ('.png','.jpg','.mov','.mp4'):
                filename = file.name  # Extract filename without extension
                try:
                    parsed_time = datetime.strptime(file.stem, "%Y-%m-%d %H.%M.%S")
                    
                    image_metadata_dict[filename] = {
                        'img_url': file.as_posix(),
                        'filename': filename,
                        'folderpath':file.parent.as_posix(),
                        'file_suffix':file.suffix,
                        'timestamp': parsed_time
                    }
                    print(f"Parsed time for {filename}: {parsed_time}")
                except ValueError:
                    image_metadata_dict[filename] = {
                        'img_url': file.as_posix(),
                        'filename': filename,
                        'folderpath':file.parent.as_posix(),
                        'file_suffix':file.suffix,
                        'timestamp': None
                    }

    return image_metadata_dict


Great, now it's working, I could update an  image into my google photo so great we get this MVP

Now, I would like to make the project more modular, Having said that, I would like prove this "MVP" into a 

In [3]:

base_path = Path(os.environ['FOLDER_PATH'].replace("\\", "/"))

image_metadata_dict = {}
for folder in base_path.iterdir():
    if folder.is_dir():
        image_metadata_dict.update(create_image_metadata_dict(folder))

In [4]:
image_metadata_dict

{'2012-08-25 13.03.40.jpg': {'img_url': 'C:/Users/Facundo Toneguzzo/OneDrive/Escritorio/Dropbox/201312/Cargas de cámara (3)/2012-08-25 13.03.40.jpg',
  'filename': '2012-08-25 13.03.40.jpg',
  'folderpath': 'C:/Users/Facundo Toneguzzo/OneDrive/Escritorio/Dropbox/201312/Cargas de cámara (3)',
  'file_suffix': '.jpg',
  'timestamp': datetime.datetime(2012, 8, 25, 13, 3, 40)},
 '2012-08-25 13.03.47.jpg': {'img_url': 'C:/Users/Facundo Toneguzzo/OneDrive/Escritorio/Dropbox/201312/Cargas de cámara (3)/2012-08-25 13.03.47.jpg',
  'filename': '2012-08-25 13.03.47.jpg',
  'folderpath': 'C:/Users/Facundo Toneguzzo/OneDrive/Escritorio/Dropbox/201312/Cargas de cámara (3)',
  'file_suffix': '.jpg',
  'timestamp': datetime.datetime(2012, 8, 25, 13, 3, 47)},
 '2012-08-25 13.04.30.jpg': {'img_url': 'C:/Users/Facundo Toneguzzo/OneDrive/Escritorio/Dropbox/201312/Cargas de cámara (3)/2012-08-25 13.04.30.jpg',
  'filename': '2012-08-25 13.04.30.jpg',
  'folderpath': 'C:/Users/Facundo Toneguzzo/OneDrive/Es

In [5]:
from PIL import Image

def convert_png_to_jpg(img_name):
    im = Image.open(image_metadata_dict[img_name].get('img_url'))
    rgb_im = im.convert('RGB')
    rgb_im.save(image_metadata_dict[img_name].get('img_url').replace('.png', '.jpg'))
    image_metadata_dict[img_name]['img_url'] = image_metadata_dict[img_name].get('img_url').replace('.png', '.jpg')
    image_metadata_dict[img_name]['file_suffix'] = '.jpg'
    image_metadata_dict[img_name]['filename'] = image_metadata_dict[img_name].get('filename').replace('.png', '.jpg')




In [ ]:
import piexif

def set_exif_timestamp(img_name):
    timestamp = image_metadata_dict[img_name].get('timestamp')
    
    if not timestamp:
        print(f"[WARNING] No timestamp found for {img_name}. Skipping EXIF writing.")
        return

    img_url = image_metadata_dict[img_name]['img_url']
    formatted_time = timestamp.strftime("%Y:%m:%d %H:%M:%S")
    try:
        exif_dict = piexif.load(image_metadata_dict[img_name].get('img_url'))

        exif_dict['Exif'][piexif.ExifIFD.DateTimeOriginal] = formatted_time.encode()
        exif_dict['0th'][piexif.ImageIFD.DateTime] = formatted_time.encode()

        exif_bytes = piexif.dump(exif_dict)
        piexif.insert(exif_bytes, image_metadata_dict[img_name].get('img_url'))
    
    except Exception as e:
          print(f"[ERROR] Failed to set EXIF timestamp for {img_url}: {e}")



In [7]:
def prepare_image_for_upload(img_name):
    image_path = Path(image_metadata_dict[img_name].get('img_url'))
    timestamp = image_metadata_dict[img_name].get('timestamp', None)
    
    if image_path.suffix.lower() == '.png':
        convert_png_to_jpg(img_name)

    if timestamp:
        set_exif_timestamp(img_name)
    else:
        print(f"No valid timestamp for {image_path}")



In [8]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
import os

class GooglePhotosClient:
    def __init__(self, credentials_json_path, token_json_path, scopes):
        self.credentials_json_path = credentials_json_path
        self.token_json_path = token_json_path
        self.scopes = scopes
        self.creds = None
        self.service = None
        self.authenticate()

    def authenticate(self):
        if os.path.exists(self.token_json_path):
            self.creds = Credentials.from_authorized_user_file(self.token_json_path, self.scopes)
            print("Loaded credentials from token file.")
        else:
            flow = InstalledAppFlow.from_client_secrets_file(self.credentials_json_path, self.scopes)
            self.creds = flow.run_local_server() # if it's a pyfile flow.run_console()
            with open(self.token_json_path, 'w') as token_file:
                token_file.write(self.creds.to_json())
            print("Authentication complete. Token saved.")

        self.service = build('photoslibrary', 'v1', credentials=self.creds)

    def get_service(self):
        return self.service


In [9]:
import requests
from google.auth.transport.requests import Request

def upload_image(image_path, creds):
    if creds.expired and creds.refresh_token:
        creds.refresh(Request())
        print(f"[INFO] Refreshed access token.")
    headers = {
        'Authorization': f'Bearer {creds.token}',
        'Content-type': 'application/octet-stream',
        'X-Goog-Upload-File-Name': os.path.basename(image_path),
        'X-Goog-Upload-Protocol': 'raw',
    }

    with open(image_path, 'rb') as f:
        response = requests.post(
            url='https://photoslibrary.googleapis.com/v1/uploads',
            data=f,
            headers=headers
        )

    if response.status_code == 200:
        upload_token = response.text
        print(f"[INFO] Uploaded {image_path} - got uploadToken.")
        return upload_token
    else:
        print(f"[ERROR] Failed to upload {image_path}: {response.text}")
        return None


In [10]:
def create_media_item(upload_token, description, creds):
    service = build('photoslibrary', 'v1', credentials=creds)
    body = {
        "newMediaItems": [
            {
                "description": description,
                "simpleMediaItem": {
                    "uploadToken": upload_token
                }
            }
        ]
    }

    response = service.mediaItems().batchCreate(body=body).execute()
    return response


In [ ]:
def upload_image_with_metadata(client, img_name):
    img_url = image_metadata_dict[img_name]['img_url']
    folderpath = image_metadata_dict[img_name]['folderpath']

    print(f"[INFO] Uploading image: {img_url}")

    upload_token = upload_image(img_url, client.creds)
    if not upload_token:
        print(f"[ERROR] Skipping {img_url} due to failed upload.")
        return

    description = f"File date:{image_metadata_dict[img_name].get('timestamp')}"
    result = create_media_item(upload_token, description, client.creds)


In [12]:
# Initialize your client
client = GooglePhotosClient(
    credentials_json_path=os.environ['GOOGLE_CLIENT_SECRET_JSON'],
    token_json_path='../credentials/token.json',
    scopes=['https://www.googleapis.com/auth/photoslibrary.appendonly']
)




Loaded credentials from token file.


In [13]:
# Run the full process
for img_name in ['2016-01-07 16.05.27.jpg']:
    prepare_image_for_upload(img_name)
    upload_image_with_metadata(client, img_name)

[ERROR] Failed to set EXIF timestamp for C:/Users/Facundo Toneguzzo/OneDrive/Escritorio/Dropbox/2016/Cargas de cámara/2016-01-07 16.05.27.jpg: [Errno 2] No such file or directory: '2016-01-07 16.05.27.jpg'
[INFO] Uploading image: C:/Users/Facundo Toneguzzo/OneDrive/Escritorio/Dropbox/2016/Cargas de cámara/2016-01-07 16.05.27.jpg
[INFO] Refreshed access token.
[INFO] Uploaded C:/Users/Facundo Toneguzzo/OneDrive/Escritorio/Dropbox/2016/Cargas de cámara/2016-01-07 16.05.27.jpg - got uploadToken.
